In [1]:
# Import the library EUMDAC
import eumdac
import time
import fnmatch
import shutil

from IPython.core.display import HTML
import datetime
import xarray as xr

# Authentication

In [2]:
# Insert your personal key and secret into the single quotes

consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)
print(f"This token '{token}' expires {token.expiration}")

This token 'd4f84738-0383-32b5-9db5-901650b6f90a' expires 2022-10-26 01:30:32.641144


# Selecting a product from the Data Store

In [3]:
datastore = eumdac.DataStore(token)
selected_collection = datastore.get_collection('EO:EUM:DAT:METOP:ASCSZR1B')

In [4]:
display(HTML('<b>'+selected_collection.title+'</b>'))
display(HTML('<b>ID:</b> '+str(selected_collection)))
display(HTML('<b>Abstract:</b> '+selected_collection.abstract))

In [5]:
start = datetime.datetime(2018, 10, 30, 2, 10)

end = datetime.datetime(2018, 11, 1, 2, 10)

geometry = [[41, -71], [41, -70], [40, -70], [40, -71], [41, -71]]

In [6]:
products = selected_collection.search(
     geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
     dtstart=start,
     dtend=end)

In [7]:
# Select only one 
latest = selected_collection.search().first()
latest


<class 'eumdac.product.Product'>(EO:EUM:DAT:METOP:ASCSZR1B, ASCA_SZR_1B_M01_20221025184500Z_20221025202658Z_N_O_20221025193642Z)

# Customization

In [8]:
datatailor = eumdac.DataTailor(token)

In [9]:
chain = eumdac.tailor_models.Chain(
     product='ASCATL1SZR',
     format='netcdf4'
     )

In [10]:
customisation = datatailor.new_customisation(latest, chain)
customisation # By calling the newly created customisation we will start the customisation process

<class 'eumdac.customisation.Customisation'>(68e304ad)

In [11]:
status = "QUEUED"
sleep_time = 10 # seconds

# Customisation Loop
while status == "QUEUED" or status == "RUNNING":
    # Get the status of the ongoing customisation
    status = customisation.status

    if "DONE" in status:
        print(f"SUCCESS")
        break
    elif "ERROR" in status or 'KILLED' in status:
        print(f"UNSUCCESS, exiting")
        break
    elif "QUEUED" in status:
        print(f"QUEUED")
    elif "RUNNING" in status:
        print(f"RUNNING")
    elif "INACTIVE" in status:
        sleep_time = max(60*10, sleep_time*2)
        print(f"INACTIVE, doubling status polling time to {sleep_time} (max 10 mins)")
    time.sleep(sleep_time)

QUEUED
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
SUCCESS


# Download

In [12]:
customisation.outputs

['ASCATL1SZR--68e304ad/ASCATL1SZR_20221025T184500Z_20221025T202658Z_epct_68e304ad_F.nc']

In [13]:
nc, = fnmatch.filter(customisation.outputs, '*.nc')
nc

'ASCATL1SZR--68e304ad/ASCATL1SZR_20221025T184500Z_20221025T202658Z_epct_68e304ad_F.nc'

In [14]:
jobID= customisation._id

In [15]:
print(f"Dowloading the NetCDF output of the customisation {jobID}")

with customisation.stream_output(nc,) as stream, \
        open(stream.name, mode='wb') as fdst:
    shutil.copyfileobj(stream, fdst)

print(f"Dowloaded the NetCDF output of the customisation {jobID}")

Dowloading the NetCDF output of the customisation 68e304ad
Dowloaded the NetCDF output of the customisation 68e304ad


In [16]:
stream.name

'ASCATL1SZR_20221025T184500Z_20221025T202658Z_epct_68e304ad_F.nc'

# Open NetCDF

In [2]:
data = xr.open_dataset('ASCATL1SZR_20221025T184500Z_20221025T202658Z_epct_68e304ad_F.nc')

In [3]:
data

<xarray.Dataset>
Dimensions:              (y: 3264, x: 82, coord_grid_y: 3264, coord_grid_x: 82)
Coordinates:
    lat                  (coord_grid_y, coord_grid_x) float32 ...
    lon                  (coord_grid_y, coord_grid_x) float32 ...
Dimensions without coordinates: y, x, coord_grid_y, coord_grid_x
Data variables: (12/43)
    azi_angle_trip_fore  (y, x) float32 ...
    azi_angle_trip_mid   (y, x) float32 ...
    azi_angle_trip_aft   (y, x) float32 ...
    f_f_fore             (y, x) float32 ...
    f_f_mid              (y, x) float32 ...
    f_f_aft              (y, x) float32 ...
    ...                   ...
    num_val_trip_mid     (y, x) float32 ...
    num_val_trip_aft     (y, x) float32 ...
    sigma0_trip_fore     (y, x) float32 ...
    sigma0_trip_mid      (y, x) float32 ...
    sigma0_trip_aft      (y, x) float32 ...
    swath_indicator      (y, x) float32 ...
Attributes: (12/34)
    Conventions:               CF-1.5
    start_sensing_time:        20221025T184500Z
    stop_sensing_time:         20221025T202658.125000Z
    EPCT_geospatial_lat_max:   84.8518
    EPCT_geospatial_lat_min:   71.8730
    EPCT_geospatial_lon_max:   293.6086
    ...                        ...
    EPCT_roi:                  null
    EPCT_sensing_start:        null
    EPCT_sensing_stop:         null
    EPCT_software_ version:    3.0.2
    EPCT_software_name:        DataTailor
    EPCT_stretch_method:       null

data = xr.open_dataset(stream.name)
data